In [1]:

# %%
import sys
sys.path.append('../')


from typing import Final

# from helpers.ploting import plotBargraph, plotSubBarGraphs
from helpers.graphs import filterEdges, intersection_of_graphs

# from helpers.operator_actions import __getTrueAndNuisanceSources, __getFinalOperatorAlarmRelationGraph, getTrueAndNuisanceSourceNames
from helpers.alarms import  filterAlarmData, getDFWithCommonSourcesInAllMonths, loadAlarmsData, loadOperatorData
from helpers import alarms
import time
from datetime import datetime, timedelta
from multiprocessing import Pool
import networkx as nx

# from multiprocessing import Pool

class Action2Alarm:
    def __init__(self):
        print("class Created")

    def __checkAction_OnAlarm(self, action, alarm,time_to_react): # Ahmet Can time_to_react filter
        if action["EventTime"] >= alarm["StartTime"] and action["EventTime"] <= alarm["EndTime"] and timedelta.total_seconds(action["EventTime"]-alarm['StartTime']) <=time_to_react:
            return True
        else:
            return False

    def constructSingleAlarmsActionsGraph(self,args):  # case 3
        df_alarms  = args[0]
        df_actions = args[1]
        time_to_react = args[2] 
        g = nx.DiGraph()  # Directed graph
    
        alarm_nodes =   [alarm for alarm in df_alarms['SourceName'].unique()]
        action_nodes = ["Operator->"+action for action in df_actions['SourceName'].unique()]     

        # ---------- Adding Nodes----------------

        for node in alarm_nodes+action_nodes:
            g.add_node(node, size=0, count=0)
        
        for node in df_alarms["SourceName"]:
            g.nodes[node]["count"] +=1
        
        for node in df_actions["SourceName"]:
            g.nodes["Operator->"+node]["count"] +=1

        alarms = df_alarms.to_dict(orient="records")
        actions = df_actions.to_dict(orient="records")

        for action in actions:
            for alarm in alarms:
                if self.__checkAction_OnAlarm(action, alarm,time_to_react) == False:
                    continue
                e = ("Operator->"+action["SourceName"], alarm["SourceName"])
                if g.has_edge(e[0],e[1]) == True:
                    g.edges[e]["weight"] += 1
                else:
                    g.add_edge(e[0],e[1])
                    g.edges[e]["weight"] = 1

        return g

    def __constructMultipleAlarmsActionsGraphs(self,df_alarms, df_actions,time_to_react):
        graphs = [self.constructSingleAlarmsActionsGraph((df_alarms[df_alarms["Year-Month"].isin([month])],df_actions[df_actions["Year-Month"].isin([month])],time_to_react)) for month in  df_alarms["Year-Month"].unique()]
        return graphs
    
    def getOperatorAlarmRelationGraph(self, df_alarms, df_actions, min_graphs_intersection_filter,time_to_react):
        print(">> Finding relation between operator action and alarm")
        graphs = self.__constructMultipleAlarmsActionsGraphs(
            df_alarms, df_actions,time_to_react)
            
        assert len(graphs) == len(list(df_alarms["Year-Month"].unique()))
        
        print(f"\n\n\n\n>> Taking intersection of {len(graphs)} sub-graphs")
        G = intersection_of_graphs(graphs, min_graphs_intersection_filter)

        return G
    
    def printGraph(self,g):
        print("          ========================== start printing        ================================")
        main_g = g
        operator_nodes = [action for action in g.nodes if action.find("Operator") != -1]
        for op in operator_nodes:
            action_count = main_g.nodes[op]['count']
            num_neg = len(list(main_g.neighbors(op)))
            print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
            # s = ""
            for n in main_g.neighbors(op):
                alarm_count = main_g.nodes[n]['count']
                edge_weight = main_g.edges[(op,n)]['weight']
                action_alarms_ratio = alarm_count/action_count


                print(f"{n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})", end = ", ")
        
        print("\n           ========================== end               ================================")


    # def __getTrueAndNuisanceSources(self,df,g):
    #     source_names_requried_action = [
    #         source for source in g.nodes if source.find("Operator") == -1]
    #     df_temp = df[~df["SourceName"].isin(source_names_requried_action)]
    #     sources_not_require_action = df_temp["SourceName"].unique()
    #     return source_names_requried_action, sources_not_require_action
    

    # def getTrueAndNuisanceSourceNames(self, df_alarms,df_operator, num_sub_graphs, min_intersection_f,edge_filter):
        
    #     sources_need_action, sources_dont_need_action = self.__getTrueAndNuisanceSources(df_alarms, g=filterEdges(self.__getFinalOperatorAlarmRelationGraph(df_alarms=df_alarms, df_actions=df_operator,
    #                                                 num_graphs=num_sub_graphs, min_graphs_intersection_filter=min_intersection_f), edge_drop_factor=edge_filter))
    #     return sources_need_action, sources_dont_need_action
    
        
    # def __addOrUpdateEdge(self,g, e):
    #     if g.has_edge(*e) == True:
    #         g.edges[e]["weight"] += 1
    #     else:
    #         g.add_edge(*e, weight=1)


    #     return False



In [2]:
def foo1(args):

    df_alarms = args[0]
    df_actions =args[1]
    min_intersect = args[2]
    time_to_react = args[3]
    actionAndAlarms = []

    action2alarm = Action2Alarm()

    print(f'Months in Alarms = {df_alarms["Year-Month"].unique()} \n Months in Actions = {df_actions["Year-Month"].unique()}')

    main_g = action2alarm.getOperatorAlarmRelationGraph(df_alarms=df_alarms, df_actions=df_actions, min_graphs_intersection_filter=min_intersect, time_to_react=time_to_react)

        # %%
    operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]

    print(f">> # of Edges Main Graph {main_g.number_of_edges()}")
    print(f">> Total number of operator Tags in the graph={len(operator_nodes)}")
    print(f">> Total number of Alarm Tags in the graph = {len(main_g.nodes)- len(operator_nodes)}")

    dict_to_print = {}
    for op in operator_nodes:
        action_count = main_g.nodes[op]['count']
        num_neg = len(list(main_g.neighbors(op)))
        # print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
        s = ""
        for n in main_g.neighbors(op):
            alarm_count = main_g.nodes[n]['count']
            edge_weight = main_g.edges[(op,n)]['weight']
            # action_alarms_ratio = alarm_count/action_count

            s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), "
            dict_to_print[op] = s    
            
            # if action_count == alarm_count:
            #     s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})"
            #     dict_to_print[op] = s
            # elif action_alarms_ratio < 2 and action_alarms_ratio> 0.8 and edge_weight * 1.5> alarm_count:
            #     s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})"
            #     dict_to_print[op] = s
            # elif action_alarms_ratio >0.5 and action_alarms_ratio <=1:
            #     print(f"{n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})", end = ", ")


    for op in dict_to_print.keys():
        action_count = main_g.nodes[op]['count']
        num_neg = len(list(main_g.neighbors(op)))
        s = f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg}, => {dict_to_print[op]} " 
        # print(s)
        actionAndAlarms.append(s)
    
    return actionAndAlarms
    

In [3]:

# /home/waris/Github/research/tupras-analysis/data/processed/alarms/final/final-all-months-alarms.csv
"""  Lodading the Data and Preprocessing """

PATH = "../../data/"
# path = PATH + "/processed/alarms_with_feed/"


# PATH = "/home/waris/Github/research/tupras-analysis/data/"
alarm_file_path = PATH + "processed/alarms_with_feed/final/final-all-months-alarms-feed.csv"
op_action_file_path = PATH + "processed/operator-actions/final/final-all-month-actions.csv"

start = time.time()

df_temp_alarms =loadAlarmsData(file_path=alarm_file_path)
df_temp_actions = loadOperatorData(file_path=op_action_file_path)


""" Common Sources in all months. Try it but can be skipped. """
# df_main_alarms = getDFWithCommonSourcesInAllMonths(df_main_alarms)

""" Chaning name 2 alias for alarm data but skipping it """
# source2Alias, alias2source = convertSourceNamesToAlias(df_main_alarms)

print("Total Time to load the data ", time.time()-start)
df_temp_alarms
df_temp_alarms.info()




Total Time to load the data  23.73250961303711
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12527602 entries, 0 to 12527601
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   Message      object        
 3   Condition    object        
 4   MessageType  object        
 5   StartTime    datetime64[ns]
 6   EndTime      datetime64[ns]
 7   EndMessage   object        
 8   TimeDelta    float64       
 9   Year-Month   object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 955.8+ MB


In [4]:
df_temp_alarms = df_temp_alarms[df_temp_alarms["SourceName"].isin(["47TIC1511","47LIC1509","47LIC1503"])]

df_temp_actions = df_temp_actions[df_temp_actions["SourceName"].isin(["47FIC1502","47FIC1503","47FIC1504","47FIC1505","47FIC1523","47FIC1507"])]

df_temp_actions = df_temp_actions[df_temp_actions["Year-Month"].isin(df_temp_alarms['Year-Month'].unique())]


#actions "47FIC1502","47FIC1503","47FIC1504","47FIC1505" #,"47FIC1523","47FIC1507"

# "47FIC1502","47FIC1503","47FIC1504","47FIC1505","47FIC1523"

#alarms "47TIC1511","47LIC1503",'47LIC1509'

# df_temp_alarms = df_temp_alarms.sort_values(by='StartTime')
# df_temp_actions =  df_temp_actions.sort_values(by='EventTime')

# df_temp_actions.to_csv("temp_actions.csv", index = False)
# df_temp_alarms.to_csv("temp_alarms.csv", index = False)



In [5]:
print("Alarms >",df_temp_alarms["SourceName"].value_counts())

print("Actions >", df_temp_actions["SourceName"].value_counts())

Alarms > 47LIC1503    3087
47LIC1509     698
47TIC1511      67
Name: SourceName, dtype: int64
Actions > 47FIC1523    877
47FIC1507    459
47FIC1505    276
47FIC1504    165
47FIC1503    136
47FIC1502    135
Name: SourceName, dtype: int64


In [6]:
# temp_obj = Action2Alarm()
# test_g =  temp_obj.constructSingleAlarmsActionsGraph((df_temp_alarms,df_temp_actions,0.5*60*60))

# main_g = test_g
# # %%
# operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]

# dict_to_print = {}
# for op in operator_nodes:
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     s = ""
#     for n in main_g.neighbors(op):
#         alarm_count = main_g.nodes[n]['count']
#         edge_weight = main_g.edges[(op,n)]['weight']
#         action_alarms_ratio = alarm_count/action_count

#         s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), "
#         dict_to_print[op] = s    


# for op in dict_to_print.keys():
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg}, => {dict_to_print[op]} " )

# # temp_obj.printGraph(main_g)

In [7]:

# temp_obj = Action2Alarm()
# # test_g =  temp_obj.constructSingleAlarmsActionsGraph((df_temp_alarms,df_temp_actions,2.5*60*60))
# test_g =  temp_obj.getOperatorAlarmRelationGraph(df_alarms=df_temp_alarms, df_actions=df_temp_actions, min_graphs_intersection_filter=4, time_to_react=8*60*60)

# main_g = test_g
# operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]

# dict_to_print = {}
# for op in operator_nodes:
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     # print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
#     s = ""
#     for n in main_g.neighbors(op):
#         alarm_count = main_g.nodes[n]['count']
#         edge_weight = main_g.edges[(op,n)]['weight']
#         action_alarms_ratio = alarm_count/action_count

#         s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), "
#         dict_to_print[op] = s    

# for op in dict_to_print.keys():
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg}, => {dict_to_print[op]} " )



In [8]:
print(f"Total Alarms: {df_temp_alarms.shape}, Total Operator Action: {df_temp_actions.shape}, Total Sources: {len(df_temp_alarms['SourceName'].unique())} ")

# %%

""" 
    Filter the Alarm Data
    1. Ignore the communication Alarms
    2. Ignore the momentary alarms => 20 seconds
    3. Remove Staling Alarms => 12 hours    
    4. Remove sources which are triggered less 20 in whole dataset
    5. Include all the months
    6. DO SKIP ANY SOURCENAME IF IGNORING COMMUNICATION ALARMS
"""
ignore_comm_alarms :Final = True
momentary_alarms_f:Final = 20  # seconds
staling_alarm_f:Final = (60*60) * 12 # hours
min_alarms_per_source_f:Final = 20 # any source which is not triggered atleast 20 times in whole dataset will be removed
months_f:Final = df_temp_alarms["Year-Month"].unique()
print(f">> Unique Year-Month {months_f}")
snames_f:Final = [] # ONLY USE IF NOT IGNORING COMM ALRMS


df_alarms_new = filterAlarmData(df_temp_alarms, months=months_f, sources_filter=snames_f,
                                     monmentarly_filter=momentary_alarms_f, staling_filter=staling_alarm_f, ingore_communication_alarms=ignore_comm_alarms, min_alarms_per_source=min_alarms_per_source_f)

df_actions_new = df_temp_actions[df_temp_actions["Year-Month"].isin(months_f)]

alarms2months = {}
for sname in df_alarms_new["SourceName"].unique():
    alarms2months[sname] = df_alarms_new[df_alarms_new["SourceName"].isin([sname])]["Year-Month"].unique()

alarms2months 

monthCount2Snames = {} 

for key in range(1,len(list(df_alarms_new["Year-Month"].unique()))+1):
    monthCount2Snames[key] = []

for sname in alarms2months.keys():
    monthCount2Snames[len(alarms2months[sname])].append(sname)

print(f">> Month Count 2 alarms: {monthCount2Snames}" )

""" 
    For Opertartor Graphs
    1. Construct sub-graphs equal to number of months
    2. Min intersections is sub-graphs/2 + 1
    3. Remove edges which are not significantaly contributing to nodes count
        It will vary in op actions and in grouping. In op actions
        it will be higher but in groupign it will be lower.
"""
# num_sub_graphs:Final = len(months_f)
# min_intersection_f:Final = 2
# edge_filter:Final = 1 # BE CAREFUL OVER HERE

print("Alarms >",df_alarms_new["SourceName"].value_counts())

print("Actions >", df_actions_new["SourceName"].value_counts())

actionAndAlarms =[]

for mc in  monthCount2Snames.keys():
    if len(monthCount2Snames[mc]) == 0 or mc == 1:
        continue
    print(f"********************** Alarms Appearing exactly in {mc} months **************************" )
    actionAndAlarms += foo1((df_alarms_new[df_alarms_new['SourceName'].isin(monthCount2Snames[mc])],df_actions_new,mc//2,4*60*60))


for aa in actionAndAlarms:
    print(aa)

# foo1((df_temp_alarms,df_temp_actions,6,4*60*60))
   






Total Alarms: (3852, 10), Total Operator Action: (2048, 4), Total Sources: 3 
>> Unique Year-Month ['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 6)' '(2019, 5)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
>>Preprocessing... 
   Months to include=['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 6)' '(2019, 5)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
  Ignore Sources=[]
  Ingnore Momentarlily Alarms Filter=20seconds 
   Ignoreing Staling Alarms Filter=12.0 hours, 
 Ignore Communication Alarms = True 
 Remove sources whose count is less than 20
>> Month Count 2 alarms: {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: ['47LIC1503'], 8: [], 9: [], 10: [], 11: ['47LIC1509', '47TIC1511'], 12: []}
Alarms > 47LIC1503    830
47LIC1509    390
47TIC1511     50
Name: SourceName, dtype: int64
Actions > 47FIC1523    877
47FIC1507    459
47FIC1505    276
47FIC1504    165
47FIC1503    136
47F

In [9]:
#Operator->47FIC1507 Count:459|| Correspodning Alarms = 1, =>  47LIC1503(count:830)(weight:226),  
#Operator->47FIC1523 Count:812|| Correspodning Alarms = 1, =>  47LIC1509(count:390)(weight:411),  

In [10]:
# """
#     Plotting actions and without action alarms
# """

# true_sources, nuisance_sources = __getTrueAndNuisanceSources(df_alarms_new, main_g)

# total_number_of_alarms = df_alarms_new.shape[0]
# number_of_alarms_not_have_action = df_alarms_new[df_alarms_new["SourceName"].isin(
#     nuisance_sources)].shape[0]

# x_axis1 = ["Raw Alarms", "True Alarms", "Nuisance Alarms"]
# y_axis1 = [total_number_of_alarms, total_number_of_alarms -
#           number_of_alarms_not_have_action, number_of_alarms_not_have_action]

# x_axis2 = ["True Alarms", "Nuisance Alarms"]
# y_axis2 = [((total_number_of_alarms-number_of_alarms_not_have_action)/total_number_of_alarms)
#           * 100, (number_of_alarms_not_have_action/total_number_of_alarms)*100]


# plotSubBarGraphs(x_axis1,y_axis1,x_axis2,y_axis2)


In [11]:

# # %%

# # for debugging purposes
# assert df_alarms_new.shape[0] == (df_alarms_new[df_alarms_new["SourceName"].isin(
#     true_sources)].shape[0] + df_alarms_new[df_alarms_new["SourceName"].isin(nuisance_sources)].shape[0])

# print(f"\n>>Total number of operator Actions = {df_actions_new.shape[0]}")
# print(
#     f"\n\n>>True Sources: Total = {len(true_sources)},  Sources = {true_sources}")
# print(
#     f"\n\n>> Nuisance Sources: Total = {len(nuisance_sources)}, {nuisance_sources}")




In [12]:
# for s in nuisance_sources:
#     df_condition = df_alarms_new[df_alarms_new["SourceName"]==s]
#     print(f"{s}=> {df_condition['Condition'].unique()}")

In [13]:

# # for cross check 
# true_s, nuisance_s = getTrueAndNuisanceSourceNames(df_alarms_new,df_actions_new,num_sub_graphs,min_intersection_f,edge_filter=edge_filter)
# print(f"Cross Checking \n \n True Alarms ({len(true_s)}) = {true_s} \n\nNuisance Sources ({len(nuisance_s)}) = {nuisance_s}")



In [14]:
# import plotly.graph_objects as go

# df_true_sources = df_alarms_new.loc[df_alarms_new["SourceName"].isin(true_s)]
# df_nuisance_sources = df_alarms_new.loc[df_alarms_new["SourceName"].isin(nuisance_s)]

# month2_A_true = dict(df_true_sources["Year-Month"].value_counts())
# month2_A_nuisance = dict(df_nuisance_sources["Year-Month"].value_counts()) 






# x_axis = alarms.sortMonthYearTuple(month2_A_true.keys()) 
# y_true = [month2_A_true[month_year] for month_year in x_axis]
# trace1 = go.Bar(x=x_axis,y=y_true, name="True Alarms",text=y_true, textposition='outside' )
# y_nuisance = [month2_A_nuisance[month_year] for month_year in x_axis]
# trace2 = go.Bar(x= x_axis, y= y_nuisance,name="Nuisance Alarms",text=y_nuisance, textposition='outside')

# fig = go.Figure()
# fig.add_trace(trace1)
# fig.add_trace(trace2)


# fig.update_layout(
#     xaxis_tickfont_size=14,
#     yaxis=dict(
#         title='Number of Alarms',
#         titlefont_size=16,
#         tickfont_size=14,
#     ),
#     xaxis=dict(
#         title = "Month",
#         titlefont_size=16,
#         tickfont_size=14,
#     ),
#     legend=dict(
#         x=0,
#         y=1.0,
#         bgcolor='rgba(255, 255, 255, 0)',
#         bordercolor='rgba(255, 255, 255, 0)'
#     ),
#     barmode='group',
#     height=600, 
#     width=1100,
#     template='seaborn', # ggplot2
#     # bargap=0. # gap between bars of adjacent location coordinates.
#     bargroupgap=0.1 # gap between bars of the same location coordinate.
# )

# fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',textfont_size=80)
# # fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.show()





# # %%


In [15]:
# main_g["47XL016-ANN"]